In [27]:
%load_ext autoreload
%autoreload 2

import sys
import os
import re
import pandas as pd
from tqdm.notebook import tqdm
from elasticsearch import Elasticsearch

module_path = os.path.dirname(os.getcwd())
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import load_pkl_file
from utils.elastic_utils import connect_es, run_question_query
from utils.model_utils import DocumentReader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Example Notebook

The purpose of this notebook is to demonstrate how to work with NQ data in Elasticsearch and assumes that `prepare_data.py` has been executed per the repo's README.md.

## Connect to Local ElasticSearch Instance

In [16]:
# assumes ElasticSearch is already running
es = connect_es(host='localhost', port=9200)

In [17]:
es.ping()

True

### Load NQ Question/Answer Records 

In [19]:
# load data
qa_records = load_pkl_file('../data/eval_data/qa_records_fullsys.pkl')

In [21]:
qa_records[0]

{'example_id': 5328212470870865242,
 'document_title': 'The_Mother_(How_I_Met_Your_Mother)',
 'document_url': 'https://en.wikipedia.org//w/index.php?title=The_Mother_(How_I_Met_Your_Mother)&amp;oldid=802354471',
 'question_text': 'how i.met your mother who is the mother',
 'short_answer': 'Tracy McConnell'}

### Ask a Question to ElasticSearch

In [25]:
question = 'Who is the mother in how i met your mother?'

res = run_question_query(es_obj=es,
                           index_name='demo_index',
                           question_text=question,
                           n_results=5)

In [26]:
res

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 46, 'relation': 'eq'},
  'max_score': 15.704055,
  'hits': [{'_index': 'demo_index',
    '_type': '_doc',
    '_id': '0',
    '_score': 15.704055,
    '_source': {'document_title': 'The_Mother_(How_I_Met_Your_Mother)',
     'document_url': 'https://en.wikipedia.org//w/index.php?title=The_Mother_(How_I_Met_Your_Mother)&amp;oldid=802354471',
     'document_text_clean': "The Mother ( How I Met Your Mother ) - wikipedia  The Mother ( How I Met Your Mother )  Jump to : navigation , search    Tracy McConnell     How I Met Your Mother character     The Mother appearing in `` The Locket ''     First appearance   `` Lucky Penny ( unseen ) '' `` Something New '' ( seen )     Last appearance   `` Last Forever ''     Created by   Carter Bays Craig Thomas     Portrayed by   Cristin Milioti     Information     Aliases   The Mother     Gender   Female     Spouse ( s )   

### Apply BERT Reader to Get Answer

In [28]:
reader = DocumentReader("deepset/bert-base-cased-squad2")

/Users/areed/opt/anaconda3/envs/qa-retriever-eval/lib/python3.8/site-packages/transformers/tokenization_utils.py:828: FutureWarning: Parameter max_len is deprecated and will be removed in a future release. Use model_max_length instead.
  warnings.warn(


In [56]:
# lets check the first article returned

context = res['hits']['hits'][0]['_source']['document_text_clean']
reader.tokenize(question, context)

print(f'Question: {question} \n')
print(f"Answer: {reader.get_answer()}")

Question: Who is the mother in how i met your mother? 

Answer: [CLS] Who is the mother in how i met your mother ? [SEP] top of the lighthouse near Farhampton Inn . She immediately accepts . In another flashfoward in ` ` Unpause ' ' , the Mother is revealed to be pregnant with their second child , Luke , in the year 2017 . She goes into labor while she and Ted are staying at Farhampton . Name ( edit ) The Mother ' s real name is not revealed until the series finale , ` ` Last Forever ' ' . When Ted meets her at the Farhampton train station , she reveals that her name is Tracy McConnell / 
